In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Bidirectional, Dense, Dropout,
    BatchNormalization, Concatenate, GlobalMaxPooling1D,
    GlobalAveragePooling1D, Add
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
import ast

In [ ]:
# Load preprocessed datasets
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI_CW_010/banking_train_processed.csv')

test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI_CW_010/banking_test_processed.csv')

print("✅ Data loaded successfully!")
print(f"Training shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

✅ Data loaded successfully!
Training shape: (10003, 11)
Test shape: (3080, 11)


In [ ]:
# Load tokenizer
with open('/content/drive/MyDrive/Colab Notebooks/AI_CW_010/banking_tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [ ]:
# Convert tokenized sequences back to arrays
train_sequences = np.array([ast.literal_eval(seq) for seq in train_data['tokenized_sequences']])
test_sequences = np.array([ast.literal_eval(seq) for seq in test_data['tokenized_sequences']])

print(f"Training sequences shape: {train_sequences.shape}")
print(f"Test sequences shape: {test_sequences.shape}")

Training sequences shape: (10003, 14)
Test sequences shape: (3080, 14)


In [ ]:
# Prepare labels for multi-task learning
def prepare_labels(data):
    """Prepare labels for hierarchical multi-task learning"""

    # Category labels (Level 1: 4 classes)
    category_encoder = LabelEncoder()
    category_labels = category_encoder.fit_transform(data['category'])

    # Intent labels (Level 2: 77 classes)
    intent_encoder = LabelEncoder()
    intent_labels = intent_encoder.fit_transform(data['label_text'])

    # Urgency labels (Level 3: 3 classes)
    urgency_encoder = LabelEncoder()
    urgency_labels = urgency_encoder.fit_transform(data['urgency'])

    return {
        'category': category_labels,
        'intent': intent_labels,
        'urgency': urgency_labels
    }, {
        'category_encoder': category_encoder,
        'intent_encoder': intent_encoder,
        'urgency_encoder': urgency_encoder
    }


In [ ]:
# Prepare training labels
train_labels, label_encoders = prepare_labels(train_data)
test_labels, _ = prepare_labels(test_data)

print(f"Category classes: {len(label_encoders['category_encoder'].classes_)}")
print(f"Intent classes: {len(label_encoders['intent_encoder'].classes_)}")
print(f"Urgency classes: {len(label_encoders['urgency_encoder'].classes_)}")

Category classes: 4
Intent classes: 77
Urgency classes: 3


In [ ]:
# Create train/validation split from training data
X_train, X_val, y_train_cat, y_val_cat = train_test_split(
    train_sequences, train_labels['category'],
    test_size=0.2, random_state=42, stratify=train_labels['category']
)

_, _, y_train_intent, y_val_intent = train_test_split(
    train_sequences, train_labels['intent'],
    test_size=0.2, random_state=42, stratify=train_labels['category']
)

_, _, y_train_urgency, y_val_urgency = train_test_split(
    train_sequences, train_labels['urgency'],
    test_size=0.2, random_state=42, stratify=train_labels['category']
)

In [ ]:
# Organize training and validation data
train_targets = {
    'category_output': y_train_cat,
    'intent_output': y_train_intent,
    'urgency_output': y_train_urgency
}

val_targets = {
    'category_output': y_val_cat,
    'intent_output': y_val_intent,
    'urgency_output': y_val_urgency
}

print(f"Training set: {len(X_train)} samples")
print(f"Validation set: {len(X_val)} samples")


Training set: 8002 samples
Validation set: 2001 samples


In [ ]:
import pandas as pd
from google.colab import files


# Prepare training DataFrame

train_df = pd.DataFrame({
    'tokenized_sequence': [seq.tolist() for seq in X_train],
    'category_output': y_train_cat,
    'intent_output': y_train_intent,
    'urgency_output': y_train_urgency
})

# Save to CSV locally
train_csv_filename = 'train_split.csv'
train_df.to_csv(train_csv_filename, index=False)
files.download(train_csv_filename)  # download locally

# Save to Google Drive
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/AI_CW_010/' + train_csv_filename, index=False)
print(f"Training split saved locally and to Drive as {train_csv_filename}")


# Prepare validation DataFrame

val_df = pd.DataFrame({
    'tokenized_sequence': [seq.tolist() for seq in X_val],
    'category_output': y_val_cat,
    'intent_output': y_val_intent,
    'urgency_output': y_val_urgency
})

# Save to CSV locally
val_csv_filename = 'val_split.csv'
val_df.to_csv(val_csv_filename, index=False)
files.download(val_csv_filename)  # download locally

# Save to Google Drive
val_df.to_csv('/content/drive/MyDrive/Colab Notebooks/AI_CW_010/' + val_csv_filename, index=False)
print(f"Validation split saved locally and to Drive as {val_csv_filename}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Training split saved locally and to Drive as train_split.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Validation split saved locally and to Drive as val_split.csv


In [ ]:
 #Display category distribution
print(f"\nCategory distribution:")
for i, category in enumerate(label_encoders['category_encoder'].classes_):
    count = np.sum(train_labels['category'] == i)
    print(f"  {category}: {count} samples")


Category distribution:
  Account Management: 1140 samples
  Card Services: 4416 samples
  General Support: 573 samples
  Transfer & Payments: 3874 samples


# Baseline model: single-head intent classifier

In [ ]:
# ============= BASELINE MODEL CREATION =============
#print(f"\n📊 BASELINE MODEL: Single-head Intent Classifier")
#print("-" * 60)

#def build_baseline_model():
#    """Simple single-head baseline model-intent classification"""

 #   input_text = Input(shape=(max_length,), name='text_input')

    # Model Architecture
    #embedding = Embedding(vocab_size, 64, name='baseline_embedding')(input_text)
    #lstm = LSTM(64, dropout=0.3, name='baseline_lstm')(embedding)
    #dense = Dense(128, activation='relu', name='baseline_dense')(lstm)
    #dropout = Dropout(0.5, name='baseline_dropout')(dense)
    #output = Dense(77, activation='softmax', name='intent_output')(dropout)

    #baseline = Model(inputs=input_text, outputs=output, name='BaselineModel')

    #baseline.compile(
        #optimizer=Adam(learning_rate=0.001),
        #loss='sparse_categorical_crossentropy',
        #metrics=['accuracy']
    #)

    #return baseline

# Build and train baseline
#print("🔧 Building baseline model...")
#baseline_model = build_baseline_model()
#baseline_model.summary()


In [ ]:
# ============= BASELINE MODEL CREATION =============
class SingleHeadBaselineModel:
    """Simple single-head baseline for intent classification"""

    def __init__(self, vocab_size, max_length, embedding_dim=64):
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.embedding_dim = embedding_dim
        self.model = None

    def build_model(self):
        """Build simple single-head model"""

        print("Building Single-Head Baseline Model...")

        input_text = Input(shape=(self.max_length,), name='text_input')
        embedding = Embedding(self.vocab_size, self.embedding_dim, name='baseline_embedding')(input_text)
        lstm = LSTM(64, dropout=0.3, name='baseline_lstm')(embedding)
        dense = Dense(128, activation='relu', name='baseline_dense')(lstm)
        dropout = Dropout(0.5, name='baseline_dropout')(dense)
        output = Dense(77, activation='softmax', name='intent_output')(dropout)

        self.model = Model(inputs=input_text, outputs=output, name='BaselineModel')

        self.model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        print("Single-Head Baseline Model Built Successfully!")
        return self.model

    def compile_model(self, learning_rate=0.001):
        """Compile model with specified learning rate"""

        self.model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        print("Single-Head Baseline Model compiled!")
        return self.model

    def get_model_summary(self):
        """Display model architecture summary"""
        return self.model.summary()

In [ ]:
#======Build and Compile the Baseline Model========

# Model parameters
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token
max_length = train_sequences.shape[1]

print(f"\n📊 Model Parameters:")
print(f"Vocabulary size: {vocab_size}")
print(f"Max sequence length: {max_length}")
print(f"Embedding dimension: 128")

# Initialize baseline model
baseline_banking_model = SingleHeadBaselineModel(
    vocab_size=vocab_size,
    max_length=max_length,
    embedding_dim=64
)

# Build the model
baseline_model = baseline_banking_model.build_model()

# Display model summary
print("\n" + "="*60)
print("SINGLE-HEAD BASELINE MODEL ARCHITECTURE")
print("="*60)
baseline_banking_model.get_model_summary()

print(f"\nBaseline Model Summary:")
print(f"   • Input: Text sequences (length: {max_length})")
print(f"   • Architecture: Simple LSTM + Dense layers")
print(f"   • Output: Intent Classification (77 classes)")
print(f"   • Total Parameters: {baseline_model.count_params():,}")


📊 Model Parameters:
Vocabulary size: 2297
Max sequence length: 14
Embedding dimension: 128
Building Single-Head Baseline Model...
Single-Head Baseline Model Built Successfully!

SINGLE-HEAD BASELINE MODEL ARCHITECTURE


Model: "BaselineModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)         │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ baseline_embedding (Embedding)  │ (None, 14, 64)         │       147,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ baseline_lstm (LSTM)            │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ baseline_dense (Dense)          │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ baseline_dropout (Dropout)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ intent_output (Dense)           │ (None, 77)             │         9,933 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 198,285 (774.55 KB)

 Trainable params: 198,285 (774.55 KB)

 Non-trainable params: 0 (0.00 B)


Baseline Model Summary:
   • Input: Text sequences (length: 14)
   • Architecture: Simple LSTM + Dense layers
   • Output: Intent Classification (77 classes)
   • Total Parameters: 198,285


In [ ]:
# ============= TRAINING SINGLE-HEAD BASELINE MODEL =============

print(f"\n" + "="*60)
print("TRAINING SINGLE-HEAD BASELINE MODEL")
print("="*60)

print(f"Training Configuration:")
print(f"   • Architecture: Single-head LSTM for intent classification")
print(f"   • Training samples: {len(X_train)}")
print(f"   • Validation samples: {len(X_val)}")
print(f"   • Target: Intent classification only (77 classes)")
print(f"   • Optimizer: Adam with fixed learning rate")

# Start training
print(f"\nStarting Training Process...")

try:
    # Train the baseline model
    baseline_history = baseline_model.fit(
        X_train,
        y_train_intent,  # Only intent labels for single-head model
        validation_data=(X_val, y_val_intent),  # Only intent validation
        epochs=20,  # Fewer epochs for baseline
        batch_size=32,
        verbose=1
    )

    print(f"\nTraining Completed Successfully!")

    # Training summary
    final_epoch_baseline = len(baseline_history.history['loss'])
    best_val_acc_baseline = max(baseline_history.history['val_accuracy'])
    final_val_acc_baseline = baseline_history.history['val_accuracy'][-1]

    print(f"\nBaseline Training Summary:")
    print(f"   • Total epochs trained: {final_epoch_baseline}")
    print(f"   • Best validation accuracy: {best_val_acc_baseline:.4f}")
    print(f"   • Final validation accuracy: {final_val_acc_baseline:.4f}")
    print(f"   • Total parameters: {baseline_model.count_params():,}")

    # Save training history for analysis
    baseline_history_df = pd.DataFrame(baseline_history.history)
    baseline_history_df.to_csv('baseline_training_history.csv', index=False)
    print(f"   • Training history saved: baseline_training_history.csv")

except Exception as e:
    print(f"Training Error: {str(e)}")
    print("Please check your data preparation and model configuration.")

print(f"\n" + "="*60)
print("BASELINE MODEL TRAINING COMPLETE!")
print("="*60)


TRAINING SINGLE-HEAD BASELINE MODEL
Training Configuration:
   • Architecture: Single-head LSTM for intent classification
   • Training samples: 8002
   • Validation samples: 2001
   • Target: Intent classification only (77 classes)
   • Optimizer: Adam with fixed learning rate

Starting Training Process...
Epoch 1/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.0189 - loss: 4.2973 - val_accuracy: 0.0595 - val_loss: 3.7049
Epoch 2/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0847 - loss: 3.4853 - val_accuracy: 0.2119 - val_loss: 2.6705
Epoch 3/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.1835 - loss: 2.7114 - val_accuracy: 0.2814 - val_loss: 2.2806
Epoch 4/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.2535 - loss: 2.3550 - val_accuracy: 0.3698 - val_loss: 2.0101
Epoch 5/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3371 - loss: 2.0268 - val_accuracy: 0.4363 - val_loss: 1.8017
Epoch 6/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/s

In [ ]:
# ============= SAVE BASELINE MODEL TO GOOGLE DRIVE =============

if 'baseline_history' in locals():
    print(f"\nSaving Baseline Model Components to Google Drive")
    print("-" * 50)

    # Define Google Drive save directory
    save_dir = '/content/drive/MyDrive/Colab Notebooks/AI_CW_010/'

    # Save the baseline model
    baseline_model.save(f'{save_dir}baseline_single_head_model.h5')
    print("Complete baseline model saved to Google Drive")

    # Save baseline model info
    baseline_info = {
        'model_type': 'single_head_baseline',
        'architecture': 'Simple LSTM for intent classification only',
        'vocab_size': vocab_size,
        'max_length': max_length,
        'embedding_dim': 64,
        'lstm_units': 64,
        'total_parameters': baseline_model.count_params(),
        'training_epochs': final_epoch_baseline,
        'best_val_accuracy': best_val_acc_baseline,
        'final_val_accuracy': final_val_acc_baseline,
        'output_classes': 77
    }

    import json
    with open(f'{save_dir}baseline_model_info.json', 'w') as f:
        json.dump(baseline_info, f, indent=2)
    print("Baseline model info saved to Google Drive")

    # Save baseline training history
    baseline_history_df = pd.DataFrame(baseline_history.history)
    baseline_history_df.to_csv(f'{save_dir}baseline_training_history.csv', index=False)
    print("✅ Baseline training history saved to Google Drive")

    print(f"\nBaseline Model Files Saved:")
    print(f"   • baseline_single_head_model.h5")
    print(f"   • baseline_model_info.json")
    print(f"   • baseline_training_history.csv")
    print(f"   • Location: {save_dir}")

    print(f"\nBaseline Model Summary:")
    print(f"   • Model Type: Single-head intent classifier")
    print(f"   • Training Epochs: {baseline_info['training_epochs']}")

else:
    print("No trained model found. Please run training first.")


Saving Baseline Model Components to Google Drive
--------------------------------------------------
Complete baseline model saved to Google Drive
Baseline model info saved to Google Drive
✅ Baseline training history saved to Google Drive

Baseline Model Files Saved:
   • baseline_single_head_model.h5
   • baseline_model_info.json
   • baseline_training_history.csv
   • Location: /content/drive/MyDrive/Colab Notebooks/AI_CW_010/

Baseline Model Summary:
   • Model Type: Single-head intent classifier
   • Training Epochs: 20


In [ ]:
#print(f"\n Training baseline model...")
#baseline_history = baseline_model.fit(
    #X_train, y_train_intent,
    #validation_data=(X_val, y_val_intent),
    #epochs=20,  # Fewer epochs for baseline
    #batch_size=32,
    #verbose=1
#)

# Save baseline results
#baseline_val_acc = max(baseline_history.history['val_accuracy'])
#print(f"\n Baseline Results:")
#print(f"   • Architecture: Single-head LSTM")
#print(f"   • Parameters: {baseline_model.count_params():,}")
#print(f"   • Best validation accuracy: {baseline_val_acc:.4f}")

In [ ]:
# ============= SAVE BASELINE SINGLE-HEAD MODEL =============
#print(f"\n SAVING BASELINE SINGLE-HEAD MODEL")
#print("-" * 50)

# Define Google Drive save directory
#save_dir = '/content/drive/MyDrive/Colab Notebooks/AI_CW_010/'

# Save the baseline model
#baseline_model.save(f'{save_dir}baseline_single_head_model.h5')
#print(" Baseline single-head model saved to Google Drive")

# Save baseline model info
#baseline_info = {
    #'model_type': 'single_head_baseline',
    #'architecture': 'Simple LSTM for intent classification only',
    #'vocab_size': vocab_size,
    #'max_length': max_length,
    #'embedding_dim': 64,
    #'lstm_units': 64,
    #'total_parameters': baseline_model.count_params(),
    #'training_epochs': len(baseline_history.history['loss']),
    #'best_val_accuracy': max(baseline_history.history['val_accuracy']),
    #'final_val_accuracy': baseline_history.history['val_accuracy'][-1],
    #'output_classes': 77  # Intent classes only
#}

#import json
#with open(f'{save_dir}baseline_model_info.json', 'w') as f:
    #json.dump(baseline_info, f, indent=2)
#print(" Baseline model info saved to Google Drive")

# Save baseline training history
#baseline_history_df = pd.DataFrame(baseline_history.history)
#baseline_history_df.to_csv(f'{save_dir}baseline_training_history.csv', index=False)
#print(" Baseline training history saved to Google Drive")

#print(f"\n Baseline Model Summary:")
#print(f"   • Model Type: Single-head intent classifier")
#print(f"   • Training Epochs: {baseline_info['training_epochs']}")

#print(f"\n Baseline model files saved to: {save_dir}")

# Baseline 2: Basic 3-Headed Multi-Task Neural Network

In [ ]:
# ============= BASIC 3-HEADED MODEL CLASS =============

class Basic3HeadedModel:
    """Basic 3-headed model without advanced optimizations"""

    def __init__(self, vocab_size, max_length, embedding_dim=64):
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.embedding_dim = embedding_dim
        self.model = None

    def build_model(self):
        """Build basic 3-headed model"""

        print("Building Basic 3-Headed Model...")

        # Input layer
        input_text = Input(shape=(self.max_length,), name='text_input')

        # Simple embedding (smaller dimension)
        embedding = Embedding(
            input_dim=self.vocab_size,
            output_dim=self.embedding_dim,
            input_length=self.max_length,
            name='basic_embedding'
        )(input_text)

        # Single LSTM layer (not bidirectional, no dropout)
        lstm = LSTM(64, name='basic_lstm')(embedding)

        # Shared dense layer (no batch normalization, no dropout)
        shared_dense = Dense(128, activation='relu', name='shared_dense')(lstm)

        # HEAD 1: Category Classification (simple)
        category_dense = Dense(32, activation='relu', name='category_dense')(shared_dense)
        category_output = Dense(4, activation='softmax', name='category_output')(category_dense)

        # HEAD 2: Intent Classification (simple)
        intent_dense = Dense(64, activation='relu', name='intent_dense')(shared_dense)
        intent_output = Dense(77, activation='softmax', name='intent_output')(intent_dense)

        # HEAD 3: Urgency Classification (simple)
        urgency_dense = Dense(32, activation='relu', name='urgency_dense')(shared_dense)
        urgency_output = Dense(3, activation='softmax', name='urgency_output')(urgency_dense)

        # Create basic 3-headed model
        self.model = Model(
            inputs=input_text,
            outputs=[category_output, intent_output, urgency_output],
            name='Basic3HeadedModel'
        )

        print("Basic 3-Headed Model Built Successfully!")
        return self.model

    def compile_model(self, learning_rate=0.001):
        """Compile model with basic settings"""

        # Simple compilation (equal loss weights, no advanced optimization)
        self.model.compile(
            optimizer=Adam(learning_rate=learning_rate),  # Fixed learning rate
            loss={
                'category_output': 'sparse_categorical_crossentropy',
                'intent_output': 'sparse_categorical_crossentropy',
                'urgency_output': 'sparse_categorical_crossentropy'
            },
            loss_weights={  # Equal weights (no task prioritization)
                'category_output': 1.0,
                'intent_output': 1.0,
                'urgency_output': 1.0
            },
            metrics={
                'category_output': ['accuracy'],
                'intent_output': ['accuracy'],
                'urgency_output': ['accuracy']
            }
        )

        print("Basic 3-Headed Model compiled!")
        return self.model

    def get_model_summary(self):
        """Display model architecture summary"""
        return self.model.summary()


In [ ]:
# ============= BUILD BASIC 3-HEADED MODEL =============

print(f"\n" + "="*60)
print("BUILDING BASIC 3-HEADED MODEL")
print("="*60)

# Model parameters
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token
max_length = train_sequences.shape[1]

print(f"\n📊 Model Parameters:")
print(f"Vocabulary size: {vocab_size}")
print(f"Max sequence length: {max_length}")
print(f"Embedding dimension: 128")

# Initialize basic 3-headed model
basic3_banking_model = Basic3HeadedModel(
    vocab_size=vocab_size,
    max_length=max_length,
    embedding_dim=64
)

# Build the architecture
basic_3headed_model = basic3_banking_model.build_model()

# Compile with basic settings
basic_3headed_model = basic3_banking_model.compile_model(learning_rate=0.001)

# Display model architecture
print("\n" + "="*60)
print("BASIC 3-HEADED MODEL ARCHITECTURE")
print("="*60)
basic3_banking_model.get_model_summary()

print(f"\nBasic 3-Headed Model Summary:")
print(f"   • Input: Text sequences (length: {max_length})")
print(f"   • Architecture: Simple LSTM + 3 separate heads")
print(f"   • Head 1: Category Classification (4 classes)")
print(f"   • Head 2: Intent Classification (77 classes)")
print(f"   • Head 3: Urgency Classification (3 classes)")
print(f"   • Total Parameters: {basic_3headed_model.count_params():,}")
print(f"   • Optimization: Basic (no regularization, equal loss weights)")

print(f"\n" + "="*60)
print("BASIC 3-HEADED MODEL READY FOR TRAINING!")
print("="*60)


BUILDING BASIC 3-HEADED MODEL

📊 Model Parameters:
Vocabulary size: 2297
Max sequence length: 14
Embedding dimension: 128
Building Basic 3-Headed Model...
Basic 3-Headed Model Built Successfully!
Basic 3-Headed Model compiled!

BASIC 3-HEADED MODEL ARCHITECTURE


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "Basic3HeadedModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ basic_embedding     │ (None, 14, 64)    │    147,008 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ basic_lstm (LSTM)   │ (None, 64)        │     33,024 │ basic_embedding[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_dense        │ (None, 128)       │      8,320 │ basic_lstm[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_dense      │ (None, 32)        │      4,128 │ shared_dense[0][… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ intent_dense        │ (None, 64)        │      8,256 │ shared_dense[0][… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ urgency_dense       │ (None, 32)        │      4,128 │ shared_dense[0][… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_output     │ (None, 4)         │        132 │ category_dense[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ intent_output       │ (None, 77)        │      5,005 │ intent_dense[0][… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ urgency_output      │ (None, 3)         │         99 │ urgency_dense[0]… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 210,100 (820.70 KB)

 Trainable params: 210,100 (820.70 KB)

 Non-trainable params: 0 (0.00 B)


Basic 3-Headed Model Summary:
   • Input: Text sequences (length: 14)
   • Architecture: Simple LSTM + 3 separate heads
   • Head 1: Category Classification (4 classes)
   • Head 2: Intent Classification (77 classes)
   • Head 3: Urgency Classification (3 classes)
   • Total Parameters: 210,100
   • Optimization: Basic (no regularization, equal loss weights)

BASIC 3-HEADED MODEL READY FOR TRAINING!


In [ ]:
# ============= TRAINING BASIC 3-HEADED MODEL =============

print(f"\n" + "="*60)
print("TRAINING BASIC 3-HEADED MODEL")
print("="*60)

print(f"Training Configuration:")
print(f"   • Architecture: Basic 3-headed (Category + Intent + Urgency)")
print(f"   • Training samples: {len(X_train)}")
print(f"   • Validation samples: {len(X_val)}")
print(f"   • Loss weights: Equal (1.0 each)")
print(f"   • Optimizer: Adam with fixed learning rate")

# Start training
print(f"\nStarting Training Process...")

try:
    # Train the basic 3-headed model
    basic_3headed_history = basic_3headed_model.fit(
        X_train,
        train_targets,  # Multi-target dictionary
        validation_data=(X_val, val_targets),
        epochs=20,  # Fewer epochs for baseline
        batch_size=32,
        verbose=1
        # No callbacks - no early stopping, no learning rate scheduling
    )

    print(f"\nTraining Completed Successfully!")

    # Training summary
    final_epoch_basic = len(basic_3headed_history.history['loss'])
    best_val_category_acc_basic = max(basic_3headed_history.history['val_category_output_accuracy'])
    best_val_intent_acc_basic = max(basic_3headed_history.history['val_intent_output_accuracy'])
    best_val_urgency_acc_basic = max(basic_3headed_history.history['val_urgency_output_accuracy'])

    print(f"\nBasic 3-Headed Training Summary:")
    print(f"   • Total epochs trained: {final_epoch_basic}")
    print(f"   • Best validation category accuracy: {best_val_category_acc_basic:.4f}")
    print(f"   • Best validation intent accuracy: {best_val_intent_acc_basic:.4f}")
    print(f"   • Best validation urgency accuracy: {best_val_urgency_acc_basic:.4f}")
    print(f"   • Total parameters: {basic_3headed_model.count_params():,}")

    # Save training history for analysis
    basic_3headed_history_df = pd.DataFrame(basic_3headed_history.history)
    basic_3headed_history_df.to_csv('basic_3headed_training_history.csv', index=False)
    print(f"   • Training history saved: basic_3headed_training_history.csv")

except Exception as e:
    print(f"Training Error: {str(e)}")
    print("Please check your data preparation and model configuration.")

print(f"\n" + "="*60)
print("BASIC 3-HEADED MODEL TRAINING COMPLETE!")
print("="*60)


TRAINING BASIC 3-HEADED MODEL
Training Configuration:
   • Architecture: Basic 3-headed (Category + Intent + Urgency)
   • Training samples: 8002
   • Validation samples: 2001
   • Loss weights: Equal (1.0 each)
   • Optimizer: Adam with fixed learning rate

Starting Training Process...
Epoch 1/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - category_output_accuracy: 0.8376 - category_output_loss: 0.4658 - intent_output_accuracy: 0.0943 - intent_output_loss: 3.2769 - loss: 4.4694 - urgency_output_accuracy: 0.6543 - urgency_output_loss: 0.7267 - val_category_output_accuracy: 0.8656 - val_category_output_loss: 0.4017 - val_intent_output_accuracy: 0.1209 - val_intent_output_loss: 3.0655 - val_loss: 4.2086 - val_urgency_output_accuracy: 0.6472 - val_urgency_output_loss: 0.7390
Epoch 2/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - category_output_accuracy: 0.8809 - category_output_loss: 0.3387 - intent_output_accuracy: 0.1286 - intent_output_loss: 2.9369 - loss: 3.9577 - urgency_output_acc

In [ ]:
# ============= SAVE BASIC 3-HEADED MODEL TO GOOGLE DRIVE =============

if 'basic_3headed_history' in locals():
    print(f"\nSaving Basic 3-Headed Model Components to Google Drive")
    print("-" * 50)

    # Define Google Drive save directory
    save_dir = '/content/drive/MyDrive/Colab Notebooks/AI_CW_010/'

    # Save the basic 3-headed model
    basic_3headed_model.save(f'{save_dir}basic_3headed_baseline_model.h5')
    print("Complete basic 3-headed model saved to Google Drive")

    # Save basic 3-headed model info
    basic_3headed_info = {
        'model_type': 'basic_3headed_baseline',
        'architecture': 'Simple 3-headed multi-task (no advanced optimizations)',
        'vocab_size': vocab_size,
        'max_length': max_length,
        'embedding_dim': 64,
        'lstm_units': 64,
        'total_parameters': basic_3headed_model.count_params(),
        'training_epochs': final_epoch_basic,
        'best_val_category_accuracy': best_val_category_acc_basic,
        'best_val_intent_accuracy': best_val_intent_acc_basic,
        'best_val_urgency_accuracy': best_val_urgency_acc_basic,
        'optimization_level': 'basic',
        'loss_weights': 'equal (1.0 each)',
        'regularization': 'none'
    }

    import json
    with open(f'{save_dir}basic_3headed_model_info.json', 'w') as f:
        json.dump(basic_3headed_info, f, indent=2)
    print("Basic 3-headed model info saved to Google Drive")

    # Save baseline training history
    basic_3headed_history_df = pd.DataFrame(basic_3headed_history.history)
    basic_3headed_history_df.to_csv(f'{save_dir}basic_3headed_training_history.csv', index=False)
    print("✅ basic_3headed model training history saved to Google Drive")

    print(f"\nBasic 3-Headed Model Files Saved:")
    print(f"   • basic_3headed_baseline_model.h5")
    print(f"   • basic_3headed_model_info.json")
    print(f"   • basic_3headed_training_history.csv")
    print(f"   • Location: {save_dir}")

    print(f"\nBasic 3-Headed Performance Summary:")
    print(f"   • Category: {best_val_category_acc_basic:.4f}")
    print(f"   • Intent: {best_val_intent_acc_basic:.4f}")
    print(f"   • Urgency: {best_val_urgency_acc_basic:.4f}")
    print(f"   • Parameters: {basic_3headed_model.count_params():,}")

else:
    print("No trained basic 3-headed model found. Please run training first.")


Saving Basic 3-Headed Model Components to Google Drive
--------------------------------------------------
Complete basic 3-headed model saved to Google Drive
Basic 3-headed model info saved to Google Drive
✅ basic_3headed model training history saved to Google Drive

Basic 3-Headed Model Files Saved:
   • basic_3headed_baseline_model.h5
   • basic_3headed_model_info.json
   • basic_3headed_training_history.csv
   • Location: /content/drive/MyDrive/Colab Notebooks/AI_CW_010/

Basic 3-Headed Performance Summary:
   • Category: 0.9290
   • Intent: 0.7316
   • Urgency: 0.7446
   • Parameters: 210,100


# Hierarchical 3-Headed Multi-Task Neural Network

In [ ]:
import numpy as np
import random
import tensorflow as tf


# Set random seeds for reproducibility

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
#==========Defining the Hierarchical 3-Headed Multi-Task Neural Network model's blueprints (Purpose: Encapsulate everything about the model in one reusable class)===========


class Hierarchical3HeadedBankingModel:
    """Hierarchical 3-Headed Multi-Task Neural Network for Banking Automation"""

    def __init__(self, vocab_size, max_length, embedding_dim=128):
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.embedding_dim = embedding_dim
        self.model = None

    def build_model(self):
        """Build the complete 3-headed hierarchical architecture"""

        print("🏗️ Building 3-Headed Hierarchical Architecture...")

        # Input layer
        input_text = Input(shape=(self.max_length,), name='text_input')

        # Shared embedding layer
        embedding = Embedding(
            input_dim=self.vocab_size,
            output_dim=self.embedding_dim,
            input_length=self.max_length,
            mask_zero=True,
            name='shared_embedding'
        )(input_text)

        # Shared feature extraction layers
        print("   🔧 Building shared LSTM layers...")
        lstm1 = Bidirectional(
            LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3),
            name='shared_lstm1'
        )(embedding)

        lstm2 = Bidirectional(
            LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3),
            name='shared_lstm2'
        )(lstm1)

        # Global pooling for feature aggregation # concatenating both max & Ave vectors improves performance because some tasks rely on strong keywords, others on overall context.
        global_max = GlobalMaxPooling1D(name='global_max')(lstm2)
        global_avg = GlobalAveragePooling1D(name='global_avg')(lstm2)

        # Combine pooled features
        shared_features = Concatenate(name='shared_features')([global_max, global_avg])
        shared_features = BatchNormalization(name='shared_batch_norm')(shared_features)
        shared_features = Dropout(0.5, name='shared_dropout')(shared_features)

        # Dense feature representation
        shared_dense = Dense(256, activation='relu', name='shared_dense')(shared_features)
        shared_dense = BatchNormalization(name='shared_dense_bn')(shared_dense)
        shared_dense = Dropout(0.4, name='shared_dense_dropout')(shared_dense)

        # ============= HEAD 1: CATEGORY CLASSIFICATION =============
        print("   📊 Head 1: Category Classification")
        category_branch = Dense(128, activation='relu', name='category_branch')(shared_dense)
        category_branch = BatchNormalization(name='category_bn')(category_branch)
        category_branch = Dropout(0.3, name='category_dropout')(category_branch)
        category_features = Dense(64, activation='relu', name='category_features')(category_branch)
        category_output = Dense(4, activation='softmax', name='category_output')(category_features)

        # ============= HEAD 2: SPECIALIZED INTENT BRANCHES =============
        print("   🎯 Head 2: Specialized Intent Classification")

        # Card Branch
        card_branch = Dense(128, activation='relu', name='card_branch')(shared_dense)
        card_branch = BatchNormalization(name='card_bn')(card_branch)
        card_branch = Dropout(0.3, name='card_dropout')(card_branch)
        card_features = Dense(64, activation='relu', name='card_features')(card_branch)

        # Account Branch
        account_branch = Dense(128, activation='relu', name='account_branch')(shared_dense)
        account_branch = BatchNormalization(name='account_bn')(account_branch)
        account_branch = Dropout(0.3, name='account_dropout')(account_branch)
        account_features = Dense(64, activation='relu', name='account_features')(account_branch)

        # Transfer Branch
        transfer_branch = Dense(128, activation='relu', name='transfer_branch')(shared_dense)
        transfer_branch = BatchNormalization(name='transfer_bn')(transfer_branch)
        transfer_branch = Dropout(0.3, name='transfer_dropout')(transfer_branch)
        transfer_features = Dense(64, activation='relu', name='transfer_features')(transfer_branch)

        # Support Branch
        support_branch = Dense(128, activation='relu', name='support_branch')(shared_dense)
        support_branch = BatchNormalization(name='support_bn')(support_branch)
        support_branch = Dropout(0.3, name='support_dropout')(support_branch)
        support_features = Dense(64, activation='relu', name='support_features')(support_branch)

        # Combine specialized branches with hierarchical context
        hierarchical_context = Concatenate(name='hierarchical_context')([
            shared_dense, category_features  # Include category context
        ])

        # Combine all specialized features
        combined_intent_features = Concatenate(name='combined_intent_features')([
            card_features, account_features, transfer_features, support_features
        ])

        # Merge for intent prediction
        intent_context = Concatenate(name='intent_context')([
            hierarchical_context, combined_intent_features
        ])

        # Intent classification layers
        intent_dense = Dense(256, activation='relu', name='intent_dense')(intent_context)
        intent_dense = BatchNormalization(name='intent_bn')(intent_dense)
        intent_dense = Dropout(0.3, name='intent_dropout')(intent_dense)

        intent_final = Dense(128, activation='relu', name='intent_final')(intent_dense)
        intent_output = Dense(77, activation='softmax', name='intent_output')(intent_final)

        # ============= HEAD 3: URGENCY CLASSIFICATION =============
        print("   ⚡ Head 3: Urgency Classification with Hierarchical Context")

        # Use hierarchical context (shared + category + intent information)
        urgency_context = Concatenate(name='urgency_context')([
            shared_dense, category_features, intent_dense  # this use all hierarchical info
        ])

        urgency_branch = Dense(128, activation='relu', name='urgency_branch')(urgency_context)
        urgency_branch = BatchNormalization(name='urgency_bn')(urgency_branch)
        urgency_branch = Dropout(0.3, name='urgency_dropout')(urgency_branch)

        urgency_features = Dense(64, activation='relu', name='urgency_features')(urgency_branch)
        urgency_output = Dense(3, activation='softmax', name='urgency_output')(urgency_features)

        # ============= CREATE 3-HEADED MODEL =============
        self.model = Model(
            inputs=input_text,
            outputs=[category_output, intent_output, urgency_output],
            name='Hierarchical3HeadedBankingModel'
        )

        print("✅ Complete 3-Headed Hierarchical Architecture Built!")
        return self.model

    def compile_model(self, learning_rate=0.001):
        """Compile model with 3-headed multi-task loss functions"""

        # Define loss weights for balanced 3-headed learning
        loss_weights = {
            'category_output': 1.0,
            'intent_output': 2.0,
            'urgency_output': 1.5
        }

        self.model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss={
                'category_output': 'sparse_categorical_crossentropy',
                'intent_output': 'sparse_categorical_crossentropy',
                'urgency_output': 'sparse_categorical_crossentropy'
            },
            loss_weights=loss_weights,
            metrics={
                'category_output': ['accuracy'],
                'intent_output': ['accuracy'],
                'urgency_output': ['accuracy']
            }
        )

        print("✅ 3-Headed Model compiled with multi-task optimization!")
        return self.model

    def get_model_summary(self):
        """Display model architecture summary"""
        return self.model.summary()

In [ ]:
# ============= BUILD AND COMPILE 3-HEADED MODEL =============

# Model parameters
vocab_size = len(tokenizer.word_index) + 1
max_length = train_sequences.shape[1]

print(f"\n📊 Model Parameters:")
print(f"Vocabulary size: {vocab_size}")
print(f"Max sequence length: {max_length}")
print(f"Embedding dimension: 128")

# Initialize and build 3-headed model
banking_model = Hierarchical3HeadedBankingModel(
    vocab_size=vocab_size,
    max_length=max_length,
    embedding_dim=128
)

# Build the architecture
model = banking_model.build_model()

# Compile with 3-headed multi-task optimization
model = banking_model.compile_model(learning_rate=0.001)

# Display model architecture
print("\n" + "="*80)
print("COMPLETE 3-HEADED HIERARCHICAL BANKING MODEL ARCHITECTURE")
print("="*80)
banking_model.get_model_summary()

# Create model visualization
tf.keras.utils.plot_model(
    model,
    to_file='hierarchical_3headed_banking_model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=True,
    dpi=150
)

print(f"\n✅ Complete 3-Headed Hierarchical Banking Model Built!")
print("📊 Architecture Summary:")
print(f"   • Input: Text sequences (length: {max_length})")
print(f"   • Shared Features: Bidirectional LSTM + Dense layers")
print(f"   • Head 1: Category Classification (4 classes)")
print(f"   • Head 2: Intent Classification (77 classes)")
print(f"   • Head 3: Urgency Classification (3 classes)")
print(f"   • Hierarchical Context: Each head uses previous level info")
print(f"   • Total Parameters: {model.count_params():,}")


📊 Model Parameters:
Vocabulary size: 2297
Max sequence length: 14
Embedding dimension: 128
🏗️ Building 3-Headed Hierarchical Architecture...
   🔧 Building shared LSTM layers...
   📊 Head 1: Category Classification
   🎯 Head 2: Specialized Intent Classification


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:965: UserWarning: Layer 'global_max' (of type GlobalMaxPooling1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


   ⚡ Head 3: Urgency Classification with Hierarchical Context
✅ Complete 3-Headed Hierarchical Architecture Built!
✅ 3-Headed Model compiled with multi-task optimization!

COMPLETE 3-HEADED HIERARCHICAL BANKING MODEL ARCHITECTURE


Model: "Hierarchical3HeadedBankingModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_embedding    │ (None, 14, 128)   │    294,016 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, 14)        │          0 │ text_input[0][0]  │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_lstm1        │ (None, 14, 256)   │    263,168 │ shared_embedding… │
│ (Bidirectional)     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_lstm2        │ (None, 14, 128)   │    164,352 │ shared_lstm1[0][… │
│ (Bidirectional)     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max          │ (None, 128)       │          0 │ shared_lstm2[0][… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_avg          │ (None, 128)       │          0 │ shared_lstm2[0][… │
│ (GlobalAveragePool… │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_features     │ (None, 256)       │          0 │ global_max[0][0], │
│ (Concatenate)       │                   │            │ global_avg[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_batch_norm   │ (None, 256)       │      1,024 │ shared_features[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_dropout      │ (None, 256)       │          0 │ shared_batch_nor… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_dense        │ (None, 256)       │     65,792 │ shared_dropout[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_dense_bn     │ (None, 256)       │      1,024 │ shared_dense[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_dense_dropo… │ (None, 256)       │          0 │ shared_dense_bn[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_branch     │ (None, 128)       │     32,896 │ shared_dense_dro… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ card_branch (Dense) │ (None, 128)       │     32,896 │ shared_dense_dro… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ account_branch      │ (None, 128)       │     32,896 │ shared_dense_dro… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transfer_branch     │ (None, 128)       │     32,896 │ shared_dense_dro… │
│ (Dense)             │                   │            │                 

 Total params: 1,272,340 (4.85 MB)

 Trainable params: 1,269,268 (4.84 MB)

 Non-trainable params: 3,072 (12.00 KB)


✅ Complete 3-Headed Hierarchical Banking Model Built!
📊 Architecture Summary:
   • Input: Text sequences (length: 14)
   • Shared Features: Bidirectional LSTM + Dense layers
   • Head 1: Category Classification (4 classes)
   • Head 2: Intent Classification (77 classes)
   • Head 3: Urgency Classification (3 classes)
   • Hierarchical Context: Each head uses previous level info
   • Total Parameters: 1,272,340


In [ ]:
# ============= TRAINING CALLBACKS =============

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_intent_output_accuracy',  # this Monitor intent
        factor=0.5,
        patience=7,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_hierarchical_3headed_banking_model.keras',
        monitor='val_intent_output_accuracy',  #  this Save best intent performance
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    )
]

In [ ]:
# ============= MODEL TRAINING EXECUTION =============

print(f"\n" + "="*80)
print("TRAINING 3-HEADED HIERARCHICAL MULTI-TASK BANKING MODEL")
print("="*80)

print(f"📊 Training Configuration:")
print(f"   • Architecture: 3-headed hierarchical (Category + Intent + Urgency)")
print(f"   • Training samples: {len(X_train)}")
print(f"   • Validation samples: {len(X_val)}")
print(f"   • Loss weights: Category(1.0) + Intent(2.0) + Urgency(1.5)")
print(f"   • Optimizer: Adam with learning rate scheduling")

# Start training
print(f"\n🚀 Starting Training Process...")

try:
    # Train the 3-headed hierarchical multi-task model
    history = model.fit(
        X_train,
        train_targets,
        validation_data=(X_val, val_targets),
        epochs=50,
        batch_size=32,
        callbacks=callbacks,
        verbose=1
    )

    print(f"\n✅ Training Completed Successfully!")

    # Training summary
    final_epoch = len(history.history['loss'])
    best_val_intent_acc = max(history.history['val_intent_output_accuracy'])
    best_val_category_acc = max(history.history['val_category_output_accuracy'])
    best_val_urgency_acc = max(history.history['val_urgency_output_accuracy'])

    print(f"\n📊 Training Summary:")
    print(f"   • Total epochs trained: {final_epoch}")
    print(f"   • Best validation category accuracy: {best_val_category_acc:.4f}")
    print(f"   • Best validation intent accuracy: {best_val_intent_acc:.4f}")
    print(f"   • Best validation urgency accuracy: {best_val_urgency_acc:.4f}")

    # Save training history for analysis
    history_df = pd.DataFrame(history.history)
    history_df.to_csv('training_history_3headed.csv', index=False)
    print(f"   • Training history saved: training_history_3headed.csv")

except Exception as e:
    print(f"❌ Training Error: {str(e)}")
    print("Please check your data preparation and model configuration.")


TRAINING 3-HEADED HIERARCHICAL MULTI-TASK BANKING MODEL
📊 Training Configuration:
   • Architecture: 3-headed hierarchical (Category + Intent + Urgency)
   • Training samples: 8002
   • Validation samples: 2001
   • Loss weights: Category(1.0) + Intent(2.0) + Urgency(1.5)
   • Optimizer: Adam with learning rate scheduling

🚀 Starting Training Process...
Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:965: UserWarning: Layer 'global_max' (of type GlobalMaxPooling1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - category_output_accuracy: 0.5206 - category_output_loss: 1.2128 - intent_output_accuracy: 0.0350 - intent_output_loss: 4.4388 - loss: 11.8454 - urgency_output_accuracy: 0.4593 - urgency_output_loss: 1.1700
Epoch 1: val_intent_output_accuracy improved from -inf to 0.01199, saving model to best_hierarchical_3headed_banking_model.keras
251/251 ━━━━━━━━━━━━━━━━━━━━ 48s 118ms/step - category_output_accuracy: 0.5210 - category_output_loss: 1.2119 - intent_output_accuracy: 0.0351 - intent_output_loss: 4.4370 - loss: 11.8402 - urgency_output_accuracy: 0.4595 - urgency_output_loss: 1.1694 - val_category_output_accuracy: 0.5587 - val_category_output_loss: 1.2236 - val_intent_output_accuracy: 0.0120 - val_intent_output_loss: 4.6208 - val_loss: 11.8451 - val_urgency_output_accuracy: 0.5732 - val_urgency_output_loss: 0.9222 - learning_rate: 0.0010
Epoch 2/50
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - category_output_accuracy: 0.7260 - category_output_l

In [ ]:
# ============= SAVE TRAINED MODEL AND COMPONENTS =============
print(f"\n💾 SAVING TRAINED MODEL AND COMPONENTS")
print("-" * 50)

# Define Google Drive save directory
save_dir = '/content/drive/MyDrive/Colab Notebooks/AI_CW_010/'

# Create directory if it doesn't exist
import os
os.makedirs(save_dir, exist_ok=True)

# Save the complete trained model to Google Drive
#model.save(f'{save_dir}hierarchical_3headed_banking_model_complete.h5')
#print("✅ Complete model saved to Google Drive: hierarchical_3headed_banking_model_complete.h5")

# Save the complete trained model to Google Drive in .keras format
model.save(f'{save_dir}hierarchical_3headed_banking_model_complete_v2.keras')
print("✅ Complete model saved to Google Drive: hierarchical_3headed_banking_model_complete_v2.keras")

# Save label encoders to Google Drive
with open(f'{save_dir}label_encoders_3headed_v2.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)
print("✅ Label encoders saved to Google Drive: label_encoders_3headed_v2.pkl")

# Save model parameters to Google Drive
import json
model_info = {
    'vocab_size': vocab_size,
    'max_length': max_length,
    'embedding_dim': 128,
    'num_categories': len(label_encoders['category_encoder'].classes_),
    'num_intents': len(label_encoders['intent_encoder'].classes_),
    'num_urgency_levels': len(label_encoders['urgency_encoder'].classes_),
    'total_parameters': model.count_params(),
    'training_epochs': final_epoch,
    'best_val_category_accuracy': best_val_category_acc,
    'best_val_intent_accuracy': best_val_intent_acc,
    'best_val_urgency_accuracy': best_val_urgency_acc
}

with open(f'{save_dir}model_info_3headed_v2.json', 'w') as f:
    json.dump(model_info, f, indent=2)
print("✅ Model info saved to Google Drive: model_info_3headed_v2.json")

# Save training_history_3headed
training_history_3headed_df = pd.DataFrame(history.history)
training_history_3headed_df.to_csv(f'{save_dir}training_3headed_training_history_v2.csv', index=False)
print("✅ basic_3headed model training history_v2 saved to Google Drive")


# Also copy the training history to Google Drive (if not already there)
#import shutil
#if os.path.exists('training_history_3headed.csv'):
    #shutil.copy('training_history_3headed.csv', f'{save_dir}training_history_3headed.csv')
    #print("✅ Training history copied to Google Drive: training_history_3headed.csv")

print(f"\n✅ Question 2 COMPLETE - All files saved to Google Drive!")
print(f"📁 Files location: {save_dir}")


💾 SAVING TRAINED MODEL AND COMPONENTS
--------------------------------------------------
✅ Complete model saved to Google Drive: hierarchical_3headed_banking_model_complete_v2.keras
✅ Label encoders saved to Google Drive: label_encoders_3headed_v2.pkl
✅ Model info saved to Google Drive: model_info_3headed_v2.json
✅ basic_3headed model training history_v2 saved to Google Drive

✅ Question 2 COMPLETE - All files saved to Google Drive!
📁 Files location: /content/drive/MyDrive/Colab Notebooks/AI_CW_010/
